# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [57]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [58]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [59]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local', 'travel_db', 'uk_food']


In [60]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [61]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [62]:
# review a document in the establishments collection
document = db.establishments.find_one()
pprint (document)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('667dcea2edcda2cfb53df123'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None,


In [63]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [64]:
# Create a dictionary for the new restaurant data

new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "AddressLine1": "123 New Street",
    "AddressLine2": "",
    "AddressLine3": "",
    "PostCode": "AB1 2CD",
    "Phone": "0123456789",
    "Website": "http://www.newrestaurant.com",
    "RatingValue": "5",
    "LocalAuthorityBusinessID": "987654321",
    "Scores": {
        "Hygiene": 5,
        "Structural": 5,
        "ConfidenceInManagement": 5
    },
    "Geocode": {
        "longitude": 0.1278,
        "latitude": 51.5074
    },
    "RightToReply": "",
    "Distance": 0.0,
    "NewRatingPending": True
}

In [65]:
# Insert the new restaurant into the collection

establishments.insert_one(new_restaurant)

In [66]:
# Check that the new restaurant was inserted
print(establishments.find_one({"BusinessName": "Penang Flavours"}))

{'_id': ObjectId('667dda71cff6c8b620d268e2'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'AddressLine1': '123 New Street', 'AddressLine2': '', 'AddressLine3': '', 'PostCode': 'AB1 2CD', 'Phone': '0123456789', 'Website': 'http://www.newrestaurant.com', 'RatingValue': 5, 'LocalAuthorityBusinessID': '987654321', 'Scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'Geocode': {'longitude': 0.1278, 'latitude': 51.5074}, 'RightToReply': '', 'Distance': 0.0, 'NewRatingPending': True, 'BusinessTypeID': 1, 'geocode': {'longitude': None, 'latitude': None}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [67]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

query = {"BusinessType":"Restaurant/Cafe/Canteen"}
fields = {"BusinessTypeID": 1, "BusinessType": 1}
result = establishments.find_one(query, fields)
pprint(result)

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('667dcea2edcda2cfb53df123')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [68]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one({"BusinessName":"Penang Flavours"},{"$set":{"BusinessTypeID":1}})

In [69]:
# Confirm that the new restaurant was updated
pprint(establishments.find_one({"BusinessName":"Penang Flavours"}))

{'AddressLine1': '123 New Street',
 'AddressLine2': '',
 'AddressLine3': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 0.0,
 'Geocode': {'latitude': 51.5074, 'longitude': 0.1278},
 'LocalAuthorityBusinessID': '987654321',
 'NewRatingPending': True,
 'Phone': '0123456789',
 'PostCode': 'AB1 2CD',
 'RatingValue': 5,
 'RightToReply': '',
 'Scores': {'ConfidenceInManagement': 5, 'Hygiene': 5, 'Structural': 5},
 'Website': 'http://www.newrestaurant.com',
 '_id': ObjectId('667dda71cff6c8b620d268e2'),
 'geocode': {'latitude': None, 'longitude': None}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [70]:
# Find how many documents have LocalAuthorityName as "Dover"

establishments.count_documents({'LocalAuthorityName': 'Dover'})

0

In [71]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({"LocalAuthorityName":"Dover"})

In [72]:
# Check if any remaining documents include Dover
establishments.count_documents({"LocalAuthorityName":"Dover"})


0

In [73]:
# Check that other documents remain with 'find_one'
document = db.establishments.find_one()
pprint(document)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('667dcea2edcda2cfb53df123'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None,


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [74]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({},[{"$set": {
    "geocode.longitude": {"$toDouble": "$geocode.longitude"},
    "geocode.latitude": {"$toDouble": "$geocode.latitude"}
}}])

    

Use `update_many` to convert `RatingValue` to integer numbers.

In [75]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [76]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {"RatingValue": {"$not": {"$in": non_ratings}}},
    [{'$set': {"RatingValue": {'$toInt': "$RatingValue"}}}]
)

In [77]:
# Check that the coordinates and rating value are now numbers
pprint(db.establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('667dcea2edcda2cfb53df123'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None,
